In [1]:
import sys
sys.path.append("..")
import src.sim
import src.strategy
import src.clearing
from src.plot import SimulationVisualizer

In [2]:
import datetime
from typing import Dict, List, Any
from src.clearing import ClearingMechanism

class MinimalClearing(ClearingMechanism):
    """Extremely simple clearing that only matches orders with identical prices."""
    
    def clear(self, current_time: datetime.datetime, active_orders: Dict[str, Any]) -> List[Any]:
        """Match orders with identical prices for the same contract time.
        
        Args:
            current_time: Current simulation time
            active_orders: Dictionary of all active orders
            
        Returns:
            List of orders that were filled
        """
        filled_orders = []
        
        # Group orders by contract time and price
        by_contract_price = {}
        for order in active_orders.values():
            key = (order.contract_time, order.price)
            if key not in by_contract_price:
                by_contract_price[key] = {'buy': [], 'sell': []}
            by_contract_price[key][order.side].append(order)
        
        # Match orders with identical price
        for (contract_time, price), sides in by_contract_price.items():
            buys = sides['buy']
            sells = sides['sell']
            
            # If we have both buys and sells at this price, match them
            while buys and sells:
                buy = buys.pop(0)
                sell = sells.pop(0)
                
                # Set execution price
                buy.execution_price = price
                sell.execution_price = price
                
                # Add to filled orders
                filled_orders.append(buy)
                filled_orders.append(sell)
        
        return filled_orders

In [3]:
import datetime
from typing import List, Tuple
from src.strategy import Strategy

class FixedOrderStrategy(Strategy):
    """Minimal strategy that places a single order once."""
    
    def __init__(self, price=50.0, quantity=1, side='buy'):
        """Initialize with fixed order parameters."""
        super().__init__()
        self.price = price
        self.quantity = quantity
        self.side = side
        self.has_placed_order = False
    
    def update_orders(self, current_time: datetime.datetime) -> Tuple[List, List, List]:
        """Place a single order on the first call, then do nothing."""
        if not self.has_placed_order:
            # Create contract time 1 hour ahead
            contract_time = current_time + datetime.timedelta(hours=1)
            
            # Create order
            order = self.create_order(
                price=self.price,
                quantity=self.quantity,
                contract_time=contract_time,
                side=self.side
            )
            
            return [order], [], []
        else:
            return [], [], []

In [4]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from src.sim import TradingSimulation

# Setup simulation timeframe (just 3 steps)
start_time = datetime.datetime(2023, 1, 1, 10, 0)
end_time = datetime.datetime(2023, 1, 1, 15, 30)
time_step = datetime.timedelta(minutes=15)

# Create two strategies with matching prices
buy_strategy = FixedOrderStrategy(price=50.0, quantity=1, side='buy')
sell_strategy = FixedOrderStrategy(price=50.0, quantity=1, side='sell')

# Create minimal clearing mechanism
clearing = MinimalClearing()

# Setup and run simulation
sim = TradingSimulation(
    start_time=start_time,
    end_time=end_time,
    time_step=time_step,
    strategies=[buy_strategy, sell_strategy],
    clearing_mechanism=clearing
)


# After running simulation
results = sim.run()
metrics = sim.analyze()

# Create visualizer
from src.plot import SimulationVisualizer
viz = SimulationVisualizer(metrics, results)

# View all plots organized by strategy
viz.create_dashboard().show()

# Or view specific plots
viz.plot_buy_orders(strategy_id=sell_strategy.id).show()
viz.plot_sell_orders(strategy_id=sell_strategy.id).show()

# Or get specific metrics
fill_rates = sim.analyze(metrics=["fill_rate"])

# Display fill rates by strategy
print("Fill rates by strategy:")
for strategy_id, data in fill_rates["fill_rate"].items():
    print(f"{strategy_id}: {data['fill_rate']:.2%}")

# Visualize time to fill
time_to_fill = sim.analyze(metrics=["time_to_fill"])

# Create a bar chart of mean time to fill by strategy
plt.figure(figsize=(10, 6))
strategies = []
times = []

for strategy_id, data in time_to_fill["time_to_fill"].items():
    if 'mean_seconds' in data:  # Check if there were filled orders
        strategies.append(strategy_id)
        times.append(data['mean_seconds'])


# Compare execution prices
execution_prices = sim.analyze(metrics=["execution_prices"])


2025-04-17 13:38:52,160 - TradingSimulation - INFO - Starting simulation from 2023-01-01 10:00:00 to 2023-01-01 15:30:00
2025-04-17 13:38:52,164 - TradingSimulation - INFO - Simulation complete


Order history shape: (90, 12)
Total events: 90, Unique orders: 46
Event type counts:
event_type
submitted    46
filled       44
Name: count, dtype: int64
Status counts:
status
active    46
filled    44
Name: count, dtype: int64
Metrics fill_rate data:
  23d220ec-9ca7-44c8-81b2-91c120c371fa: {'submitted_orders': 23, 'filled_orders': 22, 'fill_rate': 0.9565217391304348}
  ce452162-a798-4b3d-b44f-09e09447ff11: {'submitted_orders': 23, 'filled_orders': 22, 'fill_rate': 0.9565217391304348}

Fill counts by strategy:
  23d220ec-9ca7-44c8-81b2-91c120c371fa: 22/23 (95.7% filled)
  ce452162-a798-4b3d-b44f-09e09447ff11: 22/23 (95.7% filled)


Fill rates by strategy:
23d220ec-9ca7-44c8-81b2-91c120c371fa: 95.65%
ce452162-a798-4b3d-b44f-09e09447ff11: 95.65%


<Figure size 1000x600 with 0 Axes>

In [8]:

results.keys()

Index(['order_id', 'strategy_id', 'side', 'price', 'quantity', 'contract_time',
       'submission_time', 'execution_time', 'status', 'event_type',
       'update_count', 'execution_price'],
      dtype='object')

In [5]:
results

,order_id,strategy_id,side,price,quantity,contract_time,submission_time,execution_time,status,event_type,update_count,execution_price
0,06ad1c75-2b30-467e-ae16-68adc7872083,fd2158ea-f989-42c7-94fa-61968f20890f,buy,50.0,1,2023-01-01 11:00:00,2023-01-01 10:00:00,NaT,active,submitted,0,NaN
1,7f6cf49a-8c47-4cae-aec3-2a7a79d21f2b,7f74994c-dddc-49ea-ae30-91bd2fa04b17,sell,50.0,1,2023-01-01 11:00:00,2023-01-01 10:00:00,NaT,active,submitted,0,NaN
2,06ad1c75-2b30-467e-ae16-68adc7872083,fd2158ea-f989-42c7-94fa-61968f20890f,buy,50.0,1,2023-01-01 11:00:00,2023-01-01 10:00:00,2023-01-01 10:15:00,filled,filled,0,50.0
3,7f6cf49a-8c47-4cae-aec3-2a7a79d21f2b,7f74994c-dddc-49ea-ae30-91bd2fa04b17,sell,50.0,1,2023-01-01 11:00:00,2023-01-01 10:00:00,2023-01-01 10:15:00,filled,filled,0,50.0
4,759be237-d739-4c05-a08f-08c5ecf204a2,fd2158ea-f989-42c7-94fa-61968f20890f,buy,50.0,1,2023-01-01 11:15:00,2023-01-01 10:15:00,NaT,active,submitted,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
85,1b8c3d43-9f9f-4c22-af4b-817f1f6ada54,7f74994c-dddc-49ea-ae30-91bd2fa04b17,sell,50.0,1,2023-01-01 16:15:00,2023-01-01 15:15:00,NaT,active,submitted,0,NaN
86,dd9bc4d9-7e95-4078-b0a3-a8e24ed0e3aa,fd2158ea-f989-42c7-94fa-61968f20890f,buy,50.0,1,2023-01-01 16:15:00,2023-01-01 15:15:00,2023-01-01 15:30:00,filled,filled,0,50.0
87,1b8c3d43-9f9f-4c22-af4b-817f1f6ada54,7f74994c-dddc-49ea-ae30-91bd2fa04b17,sell,50.0,1,2023-01-01 16:15:00,2023-01-01 15:15:00,2023-01-01 15:30:00,filled,filled,0,50.0
88,f50f3aa9-31c8-4396-8602-3c9ea3c7614e,fd2158ea-f989-42c7-94fa-61968f20890f,buy,50.0,1,2023-01-01 16:30:00,2023-01-01 15:30:00,NaT,active,submitted,0,NaN
